<a href="https://colab.research.google.com/github/kalapontsai/Colab/blob/main/elhomeo_%E6%A8%99%E7%B1%A4%E5%88%97%E5%8D%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

總表與空白表格都須由本機上傳一次
每張訂單可加入第3,4行(無法刪除) 編輯完成後,進行下一張訂單或者進行下載,結束程式
訂單內容以郵件文字檔複製成txt檔

In [1]:
#***********************************************
#訂單內容套入標籤貼紙 Kadelat@gmail.com
#2023/02/03 Orig Orig
#2023/02/06 V0.1 修正套表Docx檔格式,4欄12列;判斷中/大瓶,增加一張[小空瓶]標籤
#         中/大瓶標籤增加保存期限一行,日期為出貨日+4.5年
#2023/02/22 v0.2 讀入的訂單資料依照產品序號排序
#***********************************************
from google.colab import files
import os,re
import openpyxl
#Install python-docx
!pip install python-docx #<-- Yes you can directly install in Colab
#Import the tools
import docx
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT
from datetime import date
from dateutil.relativedelta import relativedelta

d1 = date.today() + relativedelta(years=+4,months=+6) #出貨日 + 4.5年
d1 = d1.strftime("%Y.%m") #含日期%Y/%m/%d

#上傳總表
if os.path.isfile('elhomeo_label.xlsx'):
    print("資料總表已存在 ...")
    fn_alldb = 'elhomeo_label.xlsx'
else:
    print("上傳總表")
    uploaded = files.upload()
    for fn_alldb in uploaded.keys():
        print('上傳檔名: "{name}" 檔案大小 {length} bytes'.format(name=fn_alldb, length=len(uploaded[fn_alldb])))
wb = openpyxl.load_workbook(fn_alldb)     # 開啟 Excel 檔案
#names = wb.sheetnames    # 讀取 Excel 裡所有工作表名稱
s1 = wb['工作表1']        # 取得工作表名稱為「工作表1」的內容
list_alldb = []
for i in range(2,s1.max_row+1):
    list_alldb.append([s1.cell(i, 1).value,s1.cell(i, 2).value,s1.cell(i, 3).value,s1.cell(i, 4).value,s1.cell(i, 5).value]) #抓取5欄資料
print('總表筆數' + ':' + str(len(list_alldb)))

#上傳標籤空白表格
if os.path.isfile('label_template.docx'):
    print("已有空白標籤檔 ...")
    fn_template = 'label_template.docx'
else:
    print("上傳空白標籤檔")
    uploaded = files.upload() #<-- Select the file you want to upload
    for fn_template in uploaded.keys():
        print('上傳檔名: "{name}" 檔案大小 {length} bytes'.format(name=fn_template, length=len(uploaded[fn_template])))

#重複訂單上傳
txt_continue = 'new'    #判斷是否為第一份訂單
txt_ans = -1    #判斷是否繼續編輯
txt_mfg = '2023/1/1'
doc_rows = 0 #doc游標橫行數
cell_count = 0 #doc游標直欄位
while txt_continue == '0' or txt_continue == 'new':
    #上傳訂單清單  來源由Gmail內容複製
    print("上傳訂單清單")
    uploaded = files.upload()
    for fn_order in uploaded.keys():
        print('上傳檔名: "{name}" 檔案大小 {length} bytes'.format(name=fn_order, length=len(uploaded[fn_order])))
    #處理gmail文字檔
    #fn_order = '001.txt' #debug mode
    f = open(fn_order)
    list_order = []
    for line in f:
        if line == '\n':
            continue
        prod = ''
        prod_sn = ''
        qty = line[1:4].strip()
        prod_catch = line[9:].strip()
        #print(qty,prod_catch)

        idx = 0
        while prod_catch[idx] != ']':
            #print(prod_catch[idx],end='')
            prod += prod_catch[idx]
            idx += 1
        #print('')
        prod = prod.replace(' ','')
        for k in list_alldb:
            if k[1] == prod:
                prod_sn = k[0]
                #print(prod_sn)
                break
        if prod_sn == '':
            print('!! [{}] 比對錯誤 !!'.format(prod))
            continue
        list_order.append([prod_sn,int(qty)])
    list_order.sort()
    print(list_order)
    f.close

    #訂單索引比對 建立標籤資料
    list_label = []
    for i in range(len(list_order)):
        for qry in list_alldb:
            if qry[0] == list_order[i][0] :
                print(qry)
                o_size = re.findall('\小瓶',qry[1])  #判斷是否為小瓶
                o_oint = re.findall('\油膏',qry[1])  #判斷是否為油膏
                #o_big = re.findall('\大瓶',qry[1])  #判斷是否為大瓶
                if len(o_size) == 0 and len(o_oint) == 0:
                    print('*中大瓶*')
                    loop_size = 1       #增加列印次數,用來列印空瓶
                    o_big = 1       #需增加保存期限的變數
                else:
                    loop_size = 0
                    o_big = 0

                while loop_size >= 0:
                    qry_qty = 0 #列印次數
                    if loop_size >= 1:
                        label_empty = '\n小空瓶'
                    else:
                        label_empty = ''
                    if o_big == 1 and loop_size == 0:   #最後一次列印(非空瓶)且物品為中或大瓶時,增加[保存期限:yyyy/mm/dd]字眼
                        label_date = '\n保存期限:' + d1
                    else:
                        label_date = '' 

                    if qry[3] != "NA" and qry[3] != "SET":
                        split_list = qry[3].replace(' ','\n')
                        while qry_qty < list_order[i][1]:   #依照數量,增加同一支標籤
                            list_label.append(qry[2] + '\n' + split_list + label_date + label_empty)
                            qry_qty += 1

                    elif qry[3] == "SET":   #該欄內容若為'SET',抓取下一欄整套組合
                        list_set = qry[4].split()
                        while qry_qty < list_order[i][1]:   #依照數量,增加整套標籤
                            list_label.append(qry[2] + label_empty) #整套氣密袋用標籤
                            if len(list_set) > 0:
                                for j in range(len(list_set)):  #逐一比對套裝組合內序號的品名
                                    for qry_set in list_alldb:
                                        if qry_set[0] == list_set[j] :
                                            split_list = qry_set[3].replace(' ','\n')
                                            list_label.append(qry_set[2] + split_list + label_date + label_empty)
                                            break
                            qry_qty += 1
                    else:
                        list_label.append(qry[2] + label_date + label_empty) # 忘了 for S001空白糖球??!!
                    loop_size -= 1
                break

    #print(list_label)
    #內容變更
    #while txt_ans != 999:
        #print('********* 標籤項次/內容 **********')
        ##row_q,line_r = divmod(len(list_label), 4) #計算顯示行列數
        #list_idx = 0
        #for i in range(len(list_label)):
            #print('[{}]{}\n'.format(list_idx,list_label[list_idx]))
            #list_idx += 1
        #print('---------- END ------------')
        #txt_ans = input('輸入變更號碼或[999]進行下一步:')
        #if not txt_ans.isdigit():
            #txt_ans = -1
        #else:
            #txt_ans = int(txt_ans)
        #if (txt_ans <= (len(list_label) - 1) and txt_ans >= 0 and #txt_ans != 999):
            #print('\n變更內容:\n{}\n'.format(list_label[txt_ans]))
            #txt_ans_1 = input('1:新增[保存期限] 2:新增[小空瓶] #其餘字元跳出:')
            #if txt_ans_1 == '1':
                #print('預設保存期限:{}'.format(txt_mfg))
                #txt_ans_mfg = input('變更日期:')
                #if txt_ans_mfg != '':
                    #list_label[txt_ans] += '\n保存期限:' + #txt_ans_mfg
                    #txt_mfg = txt_ans_mfg
                #else:
                    #list_label[txt_ans] += '\n保存期限:' + txt_mfg
            #if txt_ans_1 == '2':
                #list_label[txt_ans] += '\n小空瓶'
            #txt_ans_1 = -1

    #第一張訂單時可增加標籤紙空白區塊
    #變更doc_rows 及cell_count
    if txt_continue == 'new':
        int_space = 0
        print('標籤紙若非全新,請輸入空格(從左至右/由上至下)')
        int_space = input('輸入數字或直接Enter跳過:')
        if int_space.isdigit() and int(int_space) > 0:
            int_space = int(int_space)
            doc_rows = int_space // 4
            cell_count = int_space % 4
        doc = docx.Document(fn_template)
        tmp_cells = doc.tables[0].rows[0].cells
    #文字填入
    for i in list_label:
        #doc.tables[0].rows[doc_rows].height = 885825 #原稿高度
        tmp_cells = doc.tables[0].rows[doc_rows].cells
        tmp_cells[cell_count].text = i
        #tmp_cells[cell_count].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.RIGHT
        cell_count += 1
        if cell_count >= 4:
            doc_rows += 1
            cell_count = 0
            doc.tables[0].add_row()
    doc.add_page_break()
    style = doc.styles['Normal']
    font = style.font
    font.name = "PMingLiU"
    font.size = Pt(8)
    font.bold = True
    print('完成!!\n')
    txt_continue = input('繼續下一張訂單請輸入[0],或按Enter退出程式')
    if txt_continue == '0':
        txt_ans = -1
#處理輸出檔名
print('下載輸出標籤檔...')
save_fn = 'label_' + fn_order[:-4] + '.docx'
doc.save(save_fn)
files.download(save_fn)
print('== 結束程式 ==')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=46f37a505b885060c525bc24954c5f4b706beff570182f2d821a62754ad8590a
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx
上傳總表


Saving elhomeo_label.xlsx to elhomeo_label.xlsx
上傳檔名: "elhomeo_label.xlsx" 檔案大小 29994 bytes
總表筆數:495
上傳空白標籤檔


Saving label_template.docx to label_template.docx
上傳檔名: "label_template.docx" 檔案大小 44711 bytes
上傳訂單清單


Saving 2023022105.txt to 2023022105.txt
上傳檔名: "2023022105.txt" 檔案大小 2018 bytes
[['C0831', 1], ['C1031', 1], ['S0032', 2], ['S0082', 1], ['S0091', 1], ['S0111', 1], ['S0122', 1], ['S0131', 1], ['S0132', 1], ['S0161', 1], ['S0167', 1], ['S0221', 1], ['S0232', 1], ['S0311', 1], ['S0351', 1], ['S0391', 1], ['S0492', 1], ['S0511', 1], ['S0512', 2], ['S0552', 2], ['S0581', 1], ['S0591', 1], ['S0592', 1], ['S0599', 1], ['S0602', 1], ['S0621', 1], ['S0641', 1], ['S0651', 1], ['S0652', 1], ['S0661', 2], ['S0671', 1], ['S0678', 1], ['S0681', 1]]
['C0831', '8.氯化鈉Nat-Mur.30C中瓶', '8.氯化鈉 Nat-M. 30C', '一次5顆一天三次 或視情況增減', 'NA']
*中大瓶*
['C1031', '10.硫酸鈉NAT-Sulph.30C中瓶', '10.硫酸鈉 Nat-S. 30C', '一次5顆一天三次 或視情況增減', 'NA']
*中大瓶*
['S0032', 'AlliumCepa蔥30C大瓶', '蔥 All-C. 30C', '一次5顆一天三次 或視情況增減', 'NA']
*中大瓶*
['S0082', 'ArnicaMont山金車30C大瓶', '山金車 Arn. 30C', '一次5顆一天三次 或視情況增減', 'NA']
*中大瓶*
['S0091', 'ArsenicAlb砷30C中瓶', '砷 Ars. 30C', '一次5顆一天三次 或視情況增減', 'NA']
*中大瓶*
['S0111', 'BerberisVulgaris小蘗30C中瓶', '小蘗 Berb-V. 30C', '一

KeyboardInterrupt: ignored